In [28]:
import pandas as pd 
import pickle
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split


In [29]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [30]:
## Preprocessing the data 
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [31]:
lencode = LabelEncoder()
df['Gender'] = lencode.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [32]:
oencode = OneHotEncoder(sparse_output=False)
oencoded = oencode.fit_transform(df[['Geography']])
oencoded_df = pd.DataFrame(oencoded, columns=oencode.get_feature_names_out())
df = pd.concat([df.drop('Geography', axis=1), oencoded_df], axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [33]:
y= df['EstimatedSalary']
X = df.drop('EstimatedSalary', axis=1)

# Split into train and test data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the data 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
with open('lencode.pkl', 'wb') as file: 
    pickle.dump(lencode, file)

with open('oencode.pkl', 'wb') as file:
    pickle.dump(oencode, file)
    
with open('scaler_1.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [34]:
## ANN regression problem statement
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [35]:
# Building the model
model= Sequential([ Dense(64, activation='relu', input_shape =(X_train.shape[1],)), 
                   Dense(32, activation='relu'), 
                   Dense(1)   ## Default is linear activation 
])

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
model.summary()

c:\Python Folder\Projects\Project_10_ANN\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime 

# Set up Tensorboard

log_dir = 'regression_logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback= TensorBoard(log_dir= log_dir, histogram_freq= 1)

In [37]:
# Set up early stopping 

earlystopping_callback = EarlyStopping(monitor = 'val_loss', patience =5 , restore_best_weights = True)

In [38]:
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), 
    epochs= 100, 
    callbacks=[earlystopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 99786.0938 - mae: 99786.0938 - val_loss: 98494.9453 - val_mae: 98494.9453
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100102.8984 - mae: 100102.8984 - val_loss: 96681.8594 - val_mae: 96681.8594
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 97197.3203 - mae: 97197.3203 - val_loss: 91982.3047 - val_mae: 91982.3047
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 91950.2578 - mae: 91950.2578 - val_loss: 84294.6484 - val_mae: 84294.6484
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 84192.5391 - mae: 84192.5391 - val_loss: 74842.1562 - val_mae: 74842.1562
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 73060.4141 - mae: 73060.4141 - val_loss: 65564.7266 - val_mae: 65564.7266
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 65181.3633 - mae: 65181.3633 - val_loss: 58199.2422 - val_mae: 58199.2422
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step 

In [19]:
%load_ext tensorboard

In [25]:
%tensorboard --logdir regression_logs/fit

Reusing TensorBoard on port 6006 (pid 30664), started 0:00:17 ago. (Use '!kill 30664' to kill it.)

In [26]:
## Evaluate model on test data 
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE:{test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 51099.8711 - mae: 51099.8711
Test MAE:50319.98828125


In [27]:
model.save('regression_model.h5')